# Atividade 1 - Etapa 3 - Identificar arquivos válidos de inválidos com validação cruzada

[Proposta On-line](https://docs.google.com/document/d/e/2PACX-1vQQ2A2PZbzVLz7PA0amF7gXN4tMSoGMXbZGpYphJNjSk158CR7jZ9gQWJ12U99xalHuQvSDfF022ypu/pub)

Listar melhores metodologias de aprendizagem clássica por validação cruzada e busca de hiperparâmetros por métricas de acurácia balanceada para identificar arquivos válidos de inválidos.

In [5]:
from glob import glob

testes = glob('test/*/*.mp3')

validar = [[a, a.split('\\')[1]] for a in testes]
validar

[['test\\invalid\\a01302.mp3', 'invalid'],
 ['test\\invalid\\a01304.mp3', 'invalid'],
 ['test\\invalid\\a01322.mp3', 'invalid'],
 ['test\\invalid\\a01331.mp3', 'invalid'],
 ['test\\invalid\\a01332.mp3', 'invalid'],
 ['test\\invalid\\a01334.mp3', 'invalid'],
 ['test\\valid\\a00659.mp3', 'valid'],
 ['test\\valid\\a01291.mp3', 'valid'],
 ['test\\valid\\a01315.mp3', 'valid'],
 ['test\\valid\\a01316.mp3', 'valid'],
 ['test\\valid\\a01339.mp3', 'valid'],
 ['test\\valid\\a01346.mp3', 'valid']]

In [7]:
validar = [(a, 1 if a.split('\\')[1] == 'valid' else 0) for a in testes]
validar

[('test\\invalid\\a01302.mp3', 0),
 ('test\\invalid\\a01304.mp3', 0),
 ('test\\invalid\\a01322.mp3', 0),
 ('test\\invalid\\a01331.mp3', 0),
 ('test\\invalid\\a01332.mp3', 0),
 ('test\\invalid\\a01334.mp3', 0),
 ('test\\valid\\a00659.mp3', 1),
 ('test\\valid\\a01291.mp3', 1),
 ('test\\valid\\a01315.mp3', 1),
 ('test\\valid\\a01316.mp3', 1),
 ('test\\valid\\a01339.mp3', 1),
 ('test\\valid\\a01346.mp3', 1)]

In [9]:
# lendo mp3 e extraindo parâmetros de áudio
import librosa
import numpy as np
import pandas as pd
from tqdm import tqdm
import os

dados = []
for mp3, alvo in tqdm(validar):
    nome = os.path.basename(mp3)
    y, sr = librosa.load(mp3)
    chroma_stft = librosa.feature.chroma_stft(y=y, sr=sr)
    tonnetz = librosa.feature.tonnetz(y=y, sr=sr)
    rmse = librosa.feature.rms(y=y)
    spec_cent = librosa.feature.spectral_centroid(y=y, sr=sr)
    spec_bw = librosa.feature.spectral_bandwidth(y=y, sr=sr)
    rolloff = librosa.feature.spectral_rolloff(y=y, sr=sr)
    zcr = librosa.feature.zero_crossing_rate(y)
    mfcc = librosa.feature.mfcc(y=y, sr=sr)
    melspectrogram = librosa.feature.melspectrogram(y=y, sr=sr)
    # fazer a média dos valores
    atributos = [nome]
    atributos.extend(np.mean(chroma_stft,axis=1))
    atributos.extend(np.mean(tonnetz,axis=1))
    atributos.extend(np.mean(rmse,axis=1))
    atributos.extend(np.mean(spec_cent,axis=1))
    atributos.extend(np.mean(spec_bw,axis=1))
    atributos.extend(np.mean(rolloff,axis=1))
    atributos.extend(np.mean(zcr,axis=1))
    atributos.extend(np.mean(mfcc,axis=1))
    atributos.extend(np.mean(melspectrogram,axis=1))

    atributos.extend(np.std(chroma_stft,axis=1))
    atributos.extend(np.std(tonnetz,axis=1))
    atributos.extend(np.std(rmse,axis=1))
    atributos.extend(np.std(spec_cent,axis=1))
    atributos.extend(np.std(spec_bw,axis=1))
    atributos.extend(np.std(rolloff,axis=1))
    atributos.extend(np.std(zcr,axis=1))
    atributos.extend(np.std(mfcc,axis=1))
    atributos.extend(np.std(melspectrogram,axis=1))

    atributos.extend(np.ptp(chroma_stft,axis=1))
    atributos.extend(np.ptp(tonnetz,axis=1))
    atributos.extend(np.ptp(rmse,axis=1))
    atributos.extend(np.ptp(spec_cent,axis=1))
    atributos.extend(np.ptp(spec_bw,axis=1))
    atributos.extend(np.ptp(rolloff,axis=1))
    atributos.extend(np.ptp(zcr,axis=1))
    atributos.extend(np.ptp(mfcc,axis=1))
    atributos.extend(np.ptp(melspectrogram,axis=1))
    atributos.append(alvo)
    
    dados.append(atributos)

df = pd.DataFrame(dados)
print(df.shape)
df.head()

  0%|          | 0/12 [00:00<?, ?it/s]

100%|██████████| 12/12 [00:05<00:00,  2.13it/s]

(12, 515)


,0,1,2,3,4,5,6,7,8,9,...,505,506,507,508,509,510,511,512,513,514
0,a01302.mp3,0.544468,0.564933,0.549145,0.546134,0.530689,0.515344,0.520823,0.512898,0.518374,...,1.386801,0.521948,0.584351,0.714954,0.840160,1.132574,1.507991,0.765557,0.026926,0
1,a01304.mp3,0.460289,0.410383,0.431682,0.455560,0.449151,0.444770,0.476797,0.446563,0.429725,...,0.006144,0.004150,0.010770,0.002103,0.003895,0.014026,0.033051,0.016220,0.000165,0
2,a01322.mp3,0.591162,0.593931,0.585481,0.604381,0.587009,0.583063,0.569464,0.531537,0.535967,...,0.022621,0.012173,0.014399,0.022417,0.013952,0.007484,0.008575,0.008072,0.000673,0
3,a01331.mp3,0.353574,0.359396,0.333837,0.347284,0.375422,0.376968,0.340490,0.348802,0.368007,...,0.003713,0.004792,0.001824,0.006548,0.004671,0.005025,0.006014,0.003901,0.000584,0
4,a01332.mp3,0.342642,0.346707,0.356807,0.383393,0.349878,0.337267,0.347477,0.327970,0.325409,...,0.102426,0.059859,0.049645,0.014752,0.050645,0.097036,0.098008,0.062553,0.003945,0


In [12]:
import joblib
from sklearn.metrics import classification_report

clf = joblib.load('RandomForestClassifier.joblib')

df.set_index(0, inplace=True)
X = df.drop(514,axis=1).values
y = df[514].values

y_pred = clf.predict(X)

print(classification_report(y, y_pred))

              precision    recall  f1-score   support

           0       0.75      1.00      0.86         6
           1       1.00      0.67      0.80         6

    accuracy                           0.83        12
   macro avg       0.88      0.83      0.83        12
weighted avg       0.88      0.83      0.83        12

